In [1]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

print("POPULATING FactAirQualityDaily")

print("\nReading silver_openaq...")
df_aq = spark.table("silver_openaq")

total_records = df_aq.count()
print(f"   Silver air quality records: {total_records:,}")

print("\nFiltering for PM2.5 measurements only...")
df_pm25 = df_aq.filter(col("parameter") == "pm25")

pm25_records = df_pm25.count()
print(f"   PM2.5 records: {pm25_records:,}")

print("\nApplying data quality filters...")
print("   Removing negative PM2.5 values (physically impossible)")
print("   Removing extreme outliers (PM2.5 > 500 µg/m³)")
print("   Removing null location IDs")

issues = df_pm25.agg(
    sum(when(col("value_avg") < 0, 1).otherwise(0)).alias("negative_avg"),
    sum(when(col("value_min") < 0, 1).otherwise(0)).alias("negative_min"),
    sum(when(col("value_max") < 0, 1).otherwise(0)).alias("negative_max"),
    sum(when(col("value_avg") > 500, 1).otherwise(0)).alias("extreme_values"),
    sum(when(col("location_id").isNull(), 1).otherwise(0)).alias("null_locations")
).collect()[0]

print(f"\n   Data Quality Issues Found:")
print(f"   Records with negative avg values: {issues['negative_avg']:,}")
print(f"   Records with negative min values: {issues['negative_min']:,}")
print(f"   Records with negative max values: {issues['negative_max']:,}")
print(f"   Records with extreme values (>500): {issues['extreme_values']:,}")
print(f"   Records with null locations: {issues['null_locations']:,}")

df_pm25_clean = df_pm25.filter(
    (col("value_avg") >= 0) &
    (col("value_min") >= 0) &
    (col("value_max") >= 0) &
    (col("value_avg") <= 500) &
    (col("value_max") <= 1000) &
    (col("location_id").isNotNull())
)

clean_records = df_pm25_clean.count()
removed_records = pm25_records - clean_records
print(f"\n   After filtering: {clean_records:,} records")
print(f"   Removed: {removed_records:,} records ({(removed_records/pm25_records*100):.2f}%)")

print("\nAggregating to daily PM2.5 metrics by location...")
df_fact_aq = df_pm25_clean.groupBy(
    col("year"),
    col("month"),
    col("day"),
    col("location_id"),
    col("location_name")
).agg(
    round(avg("value_avg"), 2).alias("AvgPM25"),
    round(max("value_max"), 2).alias("MaxPM25"),
    round(min("value_min"), 2).alias("MinPM25"),
    sum("value_count").cast("int").alias("MeasurementCount")
)

df_fact_aq = df_fact_aq.withColumn(
    "AvgPM25",
    when(col("AvgPM25") < 0, 0).otherwise(col("AvgPM25"))
).withColumn(
    "MaxPM25",
    when(col("MaxPM25") < 0, 0).otherwise(col("MaxPM25"))
).withColumn(
    "MinPM25",
    when(col("MinPM25") < 0, 0).otherwise(col("MinPM25"))
)

print("\nCreating DateKey and surrogate keys...")
df_fact_aq = df_fact_aq.withColumn(
    "DateKey",
    (col("year") * 10000 + col("month") * 100 + col("day")).cast("int")
)

df_fact_aq = df_fact_aq.withColumn(
    "AirQualityDailyKey",
    row_number().over(Window.orderBy("DateKey", "location_id"))
)

df_fact_aq_final = df_fact_aq.select(
    "AirQualityDailyKey",
    "DateKey",
    col("location_id").alias("LocationID"),
    col("location_name").alias("LocationName"),
    "AvgPM25",
    "MaxPM25",
    "MinPM25",
    "MeasurementCount"
)

final_count = df_fact_aq_final.count()
print(f"   Daily aggregated PM2.5 records: {final_count:,}")

print("\nSample of cleaned data (most recent):")
df_fact_aq_final.orderBy(desc("DateKey")).show(10, truncate=False)

print("\nSaving to Lakehouse table: FactAirQualityDaily")
spark.sql("DROP TABLE IF EXISTS FactAirQualityDaily")

df_fact_aq_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("FactAirQualityDaily")

print(f"   FactAirQualityDaily created with {final_count:,} records!")

print("\nValidation - Checking for data quality issues in final table...")
validation = spark.sql("""
    SELECT 
        COUNT(*) as TotalRecords,
        SUM(CASE WHEN AvgPM25 < 0 THEN 1 ELSE 0 END) as NegativeAvg,
        SUM(CASE WHEN MinPM25 < 0 THEN 1 ELSE 0 END) as NegativeMin,
        SUM(CASE WHEN MaxPM25 < 0 THEN 1 ELSE 0 END) as NegativeMax,
        ROUND(MIN(MinPM25), 2) as MinValue,
        ROUND(MAX(MaxPM25), 2) as MaxValue,
        ROUND(AVG(AvgPM25), 2) as OverallAvgPM25
    FROM FactAirQualityDaily
""")

print("\nFinal Data Quality Check:")
validation.show(truncate=False)

print("SUMMARY BY YEAR AND LOCATION")

spark.sql("""
    SELECT 
        CAST(DateKey / 10000 AS INT) as Year,
        LocationName,
        COUNT(DISTINCT DateKey) as DaysWithMeasurements,
        ROUND(AVG(AvgPM25), 2) as AvgPM25Overall,
        ROUND(MAX(MaxPM25), 2) as MaxPM25Overall,
        SUM(MeasurementCount) as TotalMeasurements
    FROM FactAirQualityDaily
    WHERE AvgPM25 IS NOT NULL
    GROUP BY CAST(DateKey / 10000 AS INT), LocationName
    ORDER BY Year, LocationName
""").show(100, truncate=False)

print("FACT TABLE CREATION COMPLETE - DATA QUALITY VALIDATED")

print(f"""
SUMMARY:
Total records created: {final_count:,}
Records removed due to quality issues: {removed_records:,}
Quality filters applied:
  PM2.5 values >= 0 (no negative values)
  PM2.5 average <= 500 µg/m³
  PM2.5 max <= 1000 µg/m³
  Valid location IDs only
""")

StatementMeta(, 05fac659-16df-4f99-8fd0-d4e0f63fa56e, 3, Finished, Available, Finished)

POPULATING FactAirQualityDaily

Reading silver_openaq...
   Silver air quality records: 4,029

Filtering for PM2.5 measurements only...
   PM2.5 records: 2,526

Applying data quality filters...
   Removing negative PM2.5 values (physically impossible)
   Removing extreme outliers (PM2.5 > 500 µg/m³)
   Removing null location IDs

   Data Quality Issues Found:
   Records with negative avg values: 0
   Records with negative min values: 334
   Records with negative max values: 0
   Records with extreme values (>500): 0
   Records with null locations: 0

   After filtering: 2,192 records
   Removed: 334 records (13.22%)

Aggregating to daily PM2.5 metrics by location...

Creating DateKey and surrogate keys...
   Daily aggregated PM2.5 records: 2,192

Sample of cleaned data (most recent):
+------------------+--------+----------+-------------------+-------+-------+-------+----------------+
|AirQualityDailyKey|DateKey |LocationID|LocationName       |AvgPM25|MaxPM25|MinPM25|MeasurementCount|
+